## Prescriptive Models and Data Analytics Problem Set #2

### Question 1

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [55]:
df = pd.read_csv("/Users/zhengyaojin/Desktop/health_data.csv")
df.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy
0,1,D,2003,0,81,0
1,2,H,2003,1,67,0
2,3,A,2003,0,54,0
3,4,E,2003,0,81,0
4,5,H,2003,0,69,0


In [56]:
# Handle the dummy variable
hos_dummies = pd.get_dummies(df['hospital_id'], drop_first=True)
hos_dummies = hos_dummies.astype(int)
health_df = pd.concat([df, hos_dummies], axis=1)
health_df.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy,B,C,D,E,F,G,H,I,J
0,1,D,2003,0,81,0,0,0,1,0,0,0,0,0,0
1,2,H,2003,1,67,0,0,0,0,0,0,0,1,0,0
2,3,A,2003,0,54,0,0,0,0,0,0,0,0,0,0
3,4,E,2003,0,81,0,0,0,0,1,0,0,0,0,0
4,5,H,2003,0,69,0,0,0,0,0,0,0,1,0,0


In [57]:
model1 = smf.ols(formula='patient_died_dummy ~ B + C + D + E + F + G + H + I + J', data=health_df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     119.3
Date:                Sun, 11 Feb 2024   Prob (F-statistic):          1.75e-220
Time:                        14:37:09   Log-Likelihood:                -7416.5
No. Observations:               24480   AIC:                         1.485e+04
Df Residuals:                   24470   BIC:                         1.493e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0970      0.006     16.355      0.0

- **Interpretation of the constant term**: The constant term in the regression output is 0.0970, which represents the predicted outcome for the response variable when all predictor variables are zero. This value indicates the foundational likelihood of mortality of the referenced hospital, hospital A, which is chosen by excluding the dummy variable in the model's initial configuration. In other words, This indicates that the baseline likelihood is 9.7% with the assumption that all other influencing factors are held constant.

- **Regression coefficient on the dummy for hospital D**: The regression coefficient of Hospital D is 0.1882. This value measures the difference in mortality risk at Hospital D compared to our baseline, Hospital A. A statistically significant positive coefficient (with a p-value below 0.05) indicates a 18.82% increase in mortality risk at Hospital D relative to Hospital A, assuming all other variables in the model remain constant.

(b)
- **Mortality Rate**: coef(D) - coef(E) = 0.1882 - (-0.0531) = 0.2413

### Question 2

In [66]:
# Filter the DataFrame for hospitals A and B, and display the first few rows
AB_data = health_df.query("hospital_id in ['A', 'B']")

AB_data.head()

,patient_id,hospital_id,admin_year,patient_died_dummy,startage,female_dummy,B,C,D,E,F,G,H,I,J
2,3,A,2003,0,54,0,0,0,0,0,0,0,0,0,0
6,7,A,2003,0,59,0,0,0,0,0,0,0,0,0,0
12,13,B,2003,0,65,0,1,0,0,0,0,0,0,0,0
15,16,A,2003,0,61,0,0,0,0,0,0,0,0,0,0
19,20,A,2003,0,71,0,0,0,0,0,0,0,0,0,0


In [59]:
model2 = smf.ols(formula = 'patient_died_dummy ~ B', data=AB_data).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9377
Date:                Sun, 11 Feb 2024   Prob (F-statistic):              0.333
Time:                        14:37:09   Log-Likelihood:                -1446.8
No. Observations:                6611   AIC:                             2898.
Df Residuals:                    6609   BIC:                             2911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0970      0.005     17.791      0.0

- **Exclusion of Key Predictors**: The model does not account for crucial predictors, like patient age at admission, which could affect both the selection of hospital and the outcome of interest, mortality. This oversight can lead to a misestimation of the sizes of the true effect.

- **Modeling Limitations Regarding Variables and Regression**: Employing the Ordinary Least Squares regression and the selected set of explanatory variables might not adequately represent the complex relationship between hospital factors and patient death rates.

- **Some Selection Bias**: The decision-making process of patients regarding their choice of hospital can be influenced by various personal criteria, which might introduce bias that could distort the analysis outcomes.

(b)
**I think it should be underestimation.**
First of all, there are variations in patient complexity across health institutions. The demographic and medical complexity of patients admitted to various healthcare facilities can differ significantly with some hospitals like tertiary or specialized services, which often deal with more critical health issues. This discrepancy in patient profiles, such as the severity of conditions treated, can naturally lead to variations in mortality outcomes, which would distort the outcome of the analysis. Without proper adjustment for such differences in patient case mix and illness severity during analytical comparisons, there's a risk of not fully capturing the true extent of mortality disparities across hospitals, which would then lead to an underestimation.

In [60]:
model = smf.ols(formula = 'patient_died_dummy ~ B + startage + female_dummy', data=AB_data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:     patient_died_dummy   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     147.6
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           1.43e-92
Time:                        14:37:09   Log-Likelihood:                -1232.8
No. Observations:                6611   AIC:                             2474.
Df Residuals:                    6607   BIC:                             2501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.1165      0.027      4.335   

**I would like to include female dummy (female_dummy) and start age (startage) in the regression.**

**female_dummy**: The disparity in mortality rates between genders can be attributed to inherent biological differences, occupational hazards that results from gender disparity, and distinct patterns in seeking medical attention. Incorporating gender-specific indicator in the analysis will allow the model to effectively adjust for gender-based differences, which should not be ignored.

**startage**: The influence of a patient's age on health outcomes cannot be overstated, as aging often correlates with an increased likelihood of chronic illnesses, cancer, and other severe health conditions associated with aging. Incorporating age into the regression can lead to a more comprehensive analysis that more precisely isolates the effects of hospital care quality on the mortality rates.

## Part 2

### Question 1

In [61]:
demand_df = pd.read_csv('/Users/zhengyaojin/Desktop/demand_data.csv')
demand_df.head()

,vendor_id,week,summer_dummy,price,sales
0,1,1,0,2.0,8788.7383
1,1,2,0,3.0,8937.9863
2,1,3,0,3.0,8740.1777
3,1,4,0,3.0,8757.1338
4,1,5,0,3.0,8739.6104


In [62]:
# fit the model for vendor 1
model_vendor_1 = smf.ols('sales ~ price', data=demand_df.query("vendor_id == 1")).fit()

print(model_vendor_1.summary())


                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.3250
Date:                Sun, 11 Feb 2024   Prob (F-statistic):              0.571
Time:                        14:37:09   Log-Likelihood:                -360.33
No. Observations:                  52   AIC:                             724.7
Df Residuals:                      50   BIC:                             728.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8983.8227    145.437     61.771      0.0

In [63]:
# Iinclude 'summer_dummy' and fit the model
model_vendor_1 = smf.ols('sales ~ price + summer_dummy', data=demand_df.query("vendor_id == 1")).fit()

print(model_vendor_1.summary())


                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.290
Method:                 Least Squares   F-statistic:                     11.42
Date:                Sun, 11 Feb 2024   Prob (F-statistic):           8.49e-05
Time:                        14:37:09   Log-Likelihood:                -350.56
No. Observations:                  52   AIC:                             707.1
Df Residuals:                      49   BIC:                             713.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     9177.5500    128.432     71.458   

Incorporating the summer_dummy variable into the sales regression model led to a shift in the price coefficient, which indicates the presence of omitted variable bias. Before this addition, the effect of price on sales appeared negligible and lacked statistical significance. However, the introduction of the summer_dummy, capturing the seasonal effects on sales, changed the price coefficient. This change in the price coefficient suggested a significant link between price and sales. By addressing this bias through the inclusion of the summer_dummy, this model now more accurately reflects the relationship between price and sales, and the increased R^2 value also illustrates such increase in the model's explanatory power.


### Question 2

In [64]:
# Fit the model for vendor 2
model_vendor_2 = smf.ols('sales ~ price', data=demand_df.query("vendor_id == 2")).fit()

print(model_vendor_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Sun, 11 Feb 2024   Prob (F-statistic):            0.00781
Time:                        14:37:09   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8411.1748    219.545     38.312      0.0

In [65]:
# Include 'summer_dummy' and fit the model
model_vendor_2 = smf.ols('sales ~ price + summer_dummy', data=demand_df.query("vendor_id == 2")).fit()

print(model_vendor_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     7.684
Date:                Sun, 11 Feb 2024   Prob (F-statistic):            0.00781
Time:                        14:37:09   Log-Likelihood:                -359.10
No. Observations:                  52   AIC:                             722.2
Df Residuals:                      50   BIC:                             726.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2105.3159     29.848     70.534   

There is either perfect or exceptionally high correlation between price and the summer_dummy variables as indicated by small eigenvalue, which suggests that there might be strong multicollinearity problems. This situation might arise if the pricing strategy aligns closely with the summer season, leading to an overlap in the information conveyed by both the price and summer_dummy variables. In specific, if prices are adjusted in a manner that directly mirrors summer seasonality, the summer_dummy variable ends up duplicating the insights offered by the price variable. Hence, including both variables in the regression will lead to problems of strong multicollinearity. 

### Question 3

**Estimation of the Price Coefficient in a Model Including Summer Dummy:**

Incorporating the summer_dummy into the model helps to adjust for seasonal variations that price changes alone do not account for. If price levels remain consistent across seasons including summer, the summer_dummy may not demonstrate a substantial correlation with price. Therefore, the estimated price coefficient in this augmented model could be very similar to the estimate from the model without the summer_dummy. 

**Estimation of the Price Coefficient in a Model Excluding Summer Dummy:**

When the summer_dummy is excluded from the regression model, the estimated price coefficient could more accurately reflect the true impact of price on sales, assuming that the model accounts for other relevant variables. If price fluctuations do not systematically align with the summer season, the exclusion of the summer_dummy is unlikely to lead to a significant omitted variable bias with respect to seasonality.

**Precision of the Price Coefficient Estimate Excluding Summer Dummy:**

Without the summer_dummy, the model's estimation of the price coefficient might suffer in precision if it fails to capture other seasonal influences on sales, leaving some of the sales variation unexplained. This omission could result in a less precise estimate of the price's effect.

**Precision of the Price Coefficient Estimate Including Summer Dummy:**
In the model that includes the summer_dummy, the precision of the price coefficient's estimate could improve. By controlling for overall seasonal shifts through the summer_dummy even in the absence of price variation that is targeted towards the summer season, this approach helps isolate the impact of price from other seasonal sales factors, which would then improve the estimate's accuracy.